# Introduction

This notebook is based on the code [network2.py](https://github.com/lmarches/neural-networks-and-deep-learning/blob/master/src/network2.py) by Michael Nielsen. 
It showcases the stochastic gradient descent learning algorithm for a feedforward neural network.

In [1]:
%matplotlib inline

# Standard library
import json
import random   
import sys
import matplotlib.pyplot as plt
# import seaborn as sns

import random

# Third-party libraries
import numpy as np
import tensorflow as tf 


# import mnist_loader
# training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Quadratic and cross-entropy cost functions

Define the two cost functions used in the optimization 

$H_{y'}(y) := - \sum_{i} ({y_i' \log(y_i) + (1-y_i') \log (1-y_i)}) $

In [2]:

class QuadraticCost(object):

    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.

        """
        return 0.5*np.linalg.norm(a-y)**2

    @staticmethod
    def delta(z, a, y):
        """Return the error delta from the output layer."""
        return (a-y) * sigmoid_prime(z)


class CrossEntropyCost(object):

    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.  Note that np.nan_to_num is used to ensure numerical
        stability.  In particular, if both ``a`` and ``y`` have a 1.0
        in the same slot, then the expression (1-y)*np.log(1-a)
        returns nan.  The np.nan_to_num ensures that that is converted
        to the correct value (0.0).

        """
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))

    @staticmethod
    def delta(z, a, y):
        """Return the error delta from the output layer.  Note that the
        parameter ``z`` is not used by the method.  It is included in
        the method's parameters in order to make the interface
        consistent with the delta method for other cost classes.

        """
        return (a-y)

# Data preparation

Data needs to be **normalised** between 0 and 1 and "packaged" in the right format (i.e. trainig data needs to be associated to their respective labels. 

Moreover, the labels needs to be converted into the output of the Neural networ (a 10 digit vector) 

In [3]:
def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

training_inputs = [np.reshape(x.astype(np.float32)/255.0,(784,1)) for x in x_train]
training_results = [vectorized_result(y) for y in y_train]
training_data_keras = zip(training_inputs, training_results)


test_inputs = [np.reshape(x.astype(np.float32)/255.0,(784,1)) for x in x_test]
test_results = [vectorized_result(y) for y in y_test]
test_data_keras = zip(test_inputs,y_test)

In [2]:
from matplotlib import pyplot

for i in range(1):
	pyplot.subplot(330 + 1 + i)
	pyplot.imshow(x_train[i], cmap=pyplot.get_cmap('gray'))
pyplot.show()

# Network architecture

This is where we define the architecture layers.  In particular: 

- The list ``sizes`` contains the number of neurons in the respective layers of the network.  
  _Example_
  
   `sizes =  [2, 3, 1]` instantiates  a three-layer network, with the first layer containing 2 neurons, the second layer 3 neurons, and the third layer 1 neuron._
- The biases and weights for the network are initialized randomly, using a Gaussian distribution with mean 0, and variance 1.  


In [5]:
#### Main Network class
class Network(object):

    def __init__(self, sizes, cost=CrossEntropyCost):

        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost=cost

# Feedforward calculation 

This is core operation of Feedforward networks: we multiply the output of previous layers with the weights of the current layer. 

Moreover we also add the bias and apply the activation function ( in this case a sigmoid)

In [6]:
class Network(Network):
    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

# Initialize weights 

Initialize each weight using a Gaussian distribution with mean 0 and standard deviation 1 over the square root of the number of weights connecting to the same neuron.  

Initialize the biases using a Gaussian distribution with mean 0 and standard deviation 1.

Note: no biases in the first layer (input layer), since biases are only ever used in computing the outputs from later layers.

In [7]:
class Network(Network):
    
    def default_weight_initializer(self):

        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def large_weight_initializer(self):

        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]


In [8]:
class Network(Network):
    
    def accuracy(self, data, convert=False):
        """Return the number of inputs in ``data`` for which the neural
        network outputs the correct result. The neural network's
        output is assumed to be the index of whichever neuron in the
        final layer has the highest activation.

        The flag ``convert`` should be set to False if the data set is
        validation or test data (the usual case), and to True if the
        data set is the training data. The need for this flag arises
        due to differences in the way the results ``y`` are
        represented in the different data sets.  In particular, it
        flags whether we need to convert between the different
        representations.  It may seem strange to use different
        representations for the different data sets.  Why not use the
        same representation for all three data sets?  It's done for
        efficiency reasons -- the program usually evaluates the cost
        on the training data and the accuracy on other data sets.
        These are different types of computations, and using different
        representations speeds things up.  More details on the
        representations can be found in
        mnist_loader.load_data_wrapper.

        """
        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                       for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in data]
        return sum(int(x == y) for (x, y) in results)

    def total_cost(self, data, lmbda, convert=False):
        """Return the total cost for the data set ``data``.  The flag
        ``convert`` should be set to False if the data set is the
        training data (the usual case), and to True if the data set is
        the validation or test data.  See comments on the similar (but
        reversed) convention for the ``accuracy`` method, above.
        """
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
        cost += 0.5*(lmbda/len(data))*sum(
            np.linalg.norm(w)**2 for w in self.weights)
        return cost

In [9]:
class Network(Network):
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda = 0.0,
            evaluation_data=None,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=False,
            monitor_training_cost=False,
            monitor_training_accuracy=False):
        """Train the neural network using mini-batch stochastic gradient
        descent.  The ``training_data`` is a list of tuples ``(x, y)``
        representing the training inputs and the desired outputs.  The
        other non-optional parameters are self-explanatory, as is the
        regularization parameter ``lmbda``.  The method also accepts
        ``evaluation_data``, usually either the validation or test
        data.  We can monitor the cost and accuracy on either the
        evaluation data or the training data, by setting the
        appropriate flags.  The method returns a tuple containing four
        lists: the (per-epoch) costs on the evaluation data, the
        accuracies on the evaluation data, the costs on the training
        data, and the accuracies on the training data.  All values are
        evaluated at the end of each training epoch.  So, for example,
        if we train for 30 epochs, then the first element of the tuple
        will be a 30-element list containing the cost on the
        evaluation data at the end of each epoch. Note that the lists
        are empty if the corresponding flag is not set.

        """
        if evaluation_data: n_data = len(evaluation_data)
        n = len(training_data)
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(
                    mini_batch, eta, lmbda, len(training_data))
            print "Epoch %s training complete" % j
            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print "Cost on training data: {}".format(cost)
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print "Accuracy on training data: {} / {}".format(
                    accuracy, n)
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print "Cost on evaluation data: {}".format(cost)
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print "Accuracy on evaluation data: {} / {}".format(
                    self.accuracy(evaluation_data), n_data)
            print
        return evaluation_cost, evaluation_accuracy, \
            training_cost, training_accuracy


In [10]:
class Network(Network):
        def backprop(self, x, y):
            """Return a tuple ``(nabla_b, nabla_w)`` representing the
            gradient for the cost function C_x.  ``nabla_b`` and
            ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
            to ``self.biases`` and ``self.weights``."""
            nabla_b = [np.zeros(b.shape) for b in self.biases]
            nabla_w = [np.zeros(w.shape) for w in self.weights]
            # feedforward
            activation = x
            activations = [x] # list to store all the activations, layer by layer
            zs = [] # list to store all the z vectors, layer by layer
            for b, w in zip(self.biases, self.weights):
                z = np.dot(w, activation)+b
                zs.append(z)
                activation = sigmoid(z)
                activations.append(activation)
            # backward pass
            delta = (self.cost).delta(zs[-1], activations[-1], y)
            nabla_b[-1] = delta
            nabla_w[-1] = np.dot(delta, activations[-2].transpose())
            # Note that the variable l in the loop below is used a little
            # differently to the notation in Chapter 2 of the book.  Here,
            # l = 1 means the last layer of neurons, l = 2 is the
            # second-last layer, and so on.  It's a renumbering of the
            # scheme in the book, used here to take advantage of the fact
            # that Python can use negative indices in lists.
            for l in xrange(2, self.num_layers):
                z = zs[-l]
                sp = sigmoid_prime(z)
                delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
                nabla_b[-l] = delta
                nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            return (nabla_b, nabla_w)


In [11]:
class Network(Network):

    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        """Update the network's weights and biases by applying gradient
        descent using backpropagation to a single mini batch.  The
        ``mini_batch`` is a list of tuples ``(x, y)``, ``eta`` is the
        learning rate, ``lmbda`` is the regularization parameter, and
        ``n`` is the total size of the training data set.

        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def save(self, filename):
        """Save the neural network to the file ``filename``."""
        data = {"sizes": self.sizes,
                "weights": [w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases],
                "cost": str(self.cost.__name__)}
        f = open(filename, "w")
        json.dump(data, f)
        f.close()


In [12]:
#### Loading a Network
def load(filename):
    """Load a neural network from the file ``filename``.  Returns an
    instance of Network.

    """
    f = open(filename, "r")
    data = json.load(f)
    f.close()
    cost = getattr(sys.modules[__name__], data["cost"])
    net = Network(data["sizes"], cost=cost)
    net.weights = [np.array(w) for w in data["weights"]]
    net.biases = [np.array(b) for b in data["biases"]]
    return net

#### Miscellaneous functions
def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the j'th position
    and zeroes elsewhere.  This is used to convert a digit (0...9)
    into a corresponding desired output from the neural network.

    """
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))


In [13]:
# net = Network([784, 10])
# net.SGD(training_data[:1000], 30, 10, 10.0, lmbda = 1000.0,
#         evaluation_data=validation_data[:100],
#         monitor_evaluation_accuracy=True)

In [14]:
# net.SGD(training_data_keras, 30, 10, 3.0, test_data=test_data_keras)

In [18]:
net = Network([784, 10])
net.SGD(training_data_keras, 30, 10, 10.0, lmbda = 10.0,
        evaluation_data=test_data_keras,
        monitor_evaluation_accuracy=True)

Epoch 0 training complete
Accuracy on evaluation data: 8354 / 10000

Epoch 1 training complete
Accuracy on evaluation data: 7871 / 10000

Epoch 2 training complete
Accuracy on evaluation data: 8349 / 10000

Epoch 3 training complete
Accuracy on evaluation data: 8157 / 10000

Epoch 4 training complete
Accuracy on evaluation data: 7840 / 10000

Epoch 5 training complete
Accuracy on evaluation data: 7920 / 10000

Epoch 6 training complete
Accuracy on evaluation data: 8221 / 10000

Epoch 7 training complete
Accuracy on evaluation data: 8115 / 10000

Epoch 8 training complete
Accuracy on evaluation data: 8631 / 10000

Epoch 9 training complete
Accuracy on evaluation data: 7811 / 10000

Epoch 10 training complete
Accuracy on evaluation data: 8356 / 10000

Epoch 11 training complete
Accuracy on evaluation data: 7869 / 10000

Epoch 12 training complete
Accuracy on evaluation data: 8086 / 10000

Epoch 13 training complete
Accuracy on evaluation data: 6732 / 10000

Epoch 14 training complete
Acc

/Users/luca/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in exp


Epoch 17 training complete
Accuracy on evaluation data: 7815 / 10000

Epoch 18 training complete
Accuracy on evaluation data: 6559 / 10000

Epoch 19 training complete
Accuracy on evaluation data: 8303 / 10000

Epoch 20 training complete
Accuracy on evaluation data: 8120 / 10000

Epoch 21 training complete
Accuracy on evaluation data: 8041 / 10000

Epoch 22 training complete
Accuracy on evaluation data: 6420 / 10000

Epoch 23 training complete
Accuracy on evaluation data: 6364 / 10000

Epoch 24 training complete
Accuracy on evaluation data: 8248 / 10000

Epoch 25 training complete
Accuracy on evaluation data: 8059 / 10000

Epoch 26 training complete
Accuracy on evaluation data: 8484 / 10000

Epoch 27 training complete
Accuracy on evaluation data: 7609 / 10000

Epoch 28 training complete
Accuracy on evaluation data: 7342 / 10000

Epoch 29 training complete
Accuracy on evaluation data: 7920 / 10000



([],
 [8354,
  7871,
  8349,
  8157,
  7840,
  7920,
  8221,
  8115,
  8631,
  7811,
  8356,
  7869,
  8086,
  6732,
  8075,
  7786,
  8325,
  7815,
  6559,
  8303,
  8120,
  8041,
  6420,
  6364,
  8248,
  8059,
  8484,
  7609,
  7342,
  7920],
 [],
 [])